In [ ]:
##Dependencies##
import folium 
import ee 
import geemap.foliumap as geemap
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

##intialize and authenticate for google earth engine
#initialize ee
ee.Authenticate() ##need to do once a week
ee.Initialize()

In [ ]:
##Function to add ee layers from class##
def add_ee_layer(ee_layer, vis_param, map_object, layer_name, attribution="Google Earth Engine"):
    image = ee.Image(ee_layer).getMapId(vis_param)["tile_fetcher"].url_format #allos us to extract the tile's URL
    print(image) #print url for tile system
    folium.raster_layers.TileLayer(
        tiles= image,  # map url
        name= layer_name, #name of layer
        overlay=True, #stack layers
        control = True,  #adds interactive layer legend
        attr=attribution, #layer source

    ).add_to(map_object) # tells which map to add layer to

In [ ]:
##Getting urban denisty raster image to create two different maps##
urban_density = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_C").first()
SD_coords = [-117.35, 32.5, -116.8, 33]

##Filtering the data by a bounding box##
bbox_sd = ee.Geometry.Rectangle(SD_coords)
urban_density_sd = urban_density.clip((bbox_sd))

LA_coords = [-118.9, 33.6, -117.6, 34.9]
bbox_la = ee.Geometry.Rectangle(LA_coords)
urban_density_la = urban_density.clip((bbox_la))



In [ ]:
tracts_df = gpd.read_file("/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/tl_2019_06_tract/tl_2019_06_tract.shp")
tracts_gdf = gpd.GeoDataFrame(tracts_df, geometry='geometry')

SD_tract_clipped = tracts_gdf.clip(SD_coords)
# Bounding Box for the contiguous United States:
#bbox = Polygon([(-135, 24), (-135, 49.8), (-66.7, 49.8), (-66.7, 24)])

# Clip our GeoDataFrames & Give it a new Coordinate Reference System (CRS):
#states_clipped = states_df.clip(bbox).to_crs("EPSG:2163")
LA_tract_clipped = tracts_gdf.clip(LA_coords)

tracts_geojsonSD = SD_tract_clipped.to_crs(epsg='4326').to_json()
tracts_geojsonLA = LA_tract_clipped.to_crs(epsg='4326').to_json()

In [ ]:
##Adding SD DUI crash Data##
DUI_SD = pd.read_csv('/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/SWITRS_SD_Alcohol.csv')

# Convert date column to datetime
DUI_SD['PROC_DATE'] = pd.to_datetime(DUI_SD['PROC_DATE'])

# Filter the DataFrame for crashes from 2017 or later
DUI_SD_filtered = DUI_SD[DUI_SD['PROC_DATE'].dt.year >= 2017]

# Create a geometry column with Point objects
DUI_SD_filtered['geometry'] = DUI_SD_filtered.apply(lambda row: Point(row.POINT_X, row.POINT_Y), axis=1)

# Create a GeoDataFrame from the DataFrame
DUI_SD_gdf = gpd.GeoDataFrame(DUI_SD_filtered, geometry='geometry')

# Filter out rows with empty geometries
DUI_SD_gdf_valid = DUI_SD_gdf[~DUI_SD_gdf.geometry.is_empty]

In [ ]:
## Creating SD interactive map##
Map_SD = folium.Map(
    location = (32.7,-117.1),
    zoom_start = 11,
    tiles="http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}",
    attr="google satelite image",
    overlay=True,
    control = True
)
#Map_SD = geemap.Map(center=[32.7,-117.1], zoom=11)
#Map_SD.add_basemap("HYBRID")

##add ee layer of SD urban density
add_ee_layer(
    ee_layer = urban_density_sd,
    vis_param = {},
    map_object=Map_SD,
    layer_name = "Urban Density",
    )

##add dui crash point in boundingbox area##
for idx, row in DUI_SD_gdf_valid.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    # Check if the point is within the bounding box
    if SD_coords[1] <= lat <= SD_coords[3] and SD_coords[0] <= lon <= SD_coords[2]:
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,  # Initial radius
            color='green',
            fill=True,
            fill_color='green',
            fill_opacity=1,
        ).add_to(Map_SD)

folium.GeoJson(
    tracts_geojsonSD,
    name='Census Tracts',
).add_to(Map_SD)

Map_SD.save("map_image.html")


In [ ]:
##Adding LA DUI crash Data##
DUI_LA = pd.read_csv('/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/SWITRS_LA_Alcohol.csv')

# Convert date column to datetime
DUI_LA['PROC_DATE'] = pd.to_datetime(DUI_LA['PROC_DATE'])

# Filter the DataFrame for crashes from 2017 or later
DUI_LA_filtered = DUI_LA[DUI_LA['PROC_DATE'].dt.year >= 2017]

# Create a geometry column with Point objects
DUI_LA_filtered['geometry'] = DUI_LA_filtered.apply(lambda row: Point(row.POINT_X, row.POINT_Y), axis=1)

# Create a GeoDataFrame from the DataFrame
DUI_LA_gdf = gpd.GeoDataFrame(DUI_LA_filtered, geometry='geometry')

# Filter out rows with empty geometries
DUI_LA_gdf_valid = DUI_LA_gdf[~DUI_LA_gdf.geometry.is_empty]

In [ ]:
## Creating LA interactive map##

#Map_LA = geemap.Map(center=(34,-118.2), zoom=10.2)
#Map_LA.add_basemap("OpenStreetMap")

# Creating SD interactive map##
Map_LA = folium.Map(
    location = (34,-118.2),
    zoom_start = 10.2,
    tiles="http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}",
    attr="google satelite image",
    overlay=True,
    control = True
)
##add ee layer of SD urban density
add_ee_layer(
    ee_layer = urban_density_la,
    vis_param = {},
    map_object=Map_LA,
    layer_name = "Urban Density"
)

##add dui crash point in boundingbox area##
for idx, row in DUI_LA_gdf_valid.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    # Check if the point is within the bounding box
    if LA_coords[1] <= lat <= LA_coords[3] and LA_coords[0] <= lon <= LA_coords[2]:
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,  # Initial radius
            color='green',
            fill=True,
            fill_color='green',
            fill_opacity=1,
        ).add_to(Map_LA)

folium.GeoJson(
    tracts_geojsonLA,
    name='Census Tracts',
).add_to(Map_LA)

Map_LA.save("LA Interactive Map.html")

